# Исследование надежности заемщиков.

Имеется датасет по клиентам кредитной организации.
Цель проекта это изучение данных, обработка пропусков, структуризация и ответ на поставленные вопросы.

· Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <br>
· Есть ли зависимость между количеством детей и возвратом кредита в срок? <br>
· Есть ли зависимость между семейным положением и возвратом кредита в срок? <br>
· Как цели кредита влияют на возврат в срок?

## Обзор данных

Импортируем библиотеку pandas и прочитаем файл с исходными данными

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')

Изучим данные, используя метод info().А также выведем первые 10 строчек на экран.

In [2]:
data.info()
data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общее количество значений в данных 21525. 
В стаже работы пропущенные и отрицательные значения.
В доходе имеются пропущенные данные.
Есть вероятность что это взаимосвязано, молодые сотрудники без стажа работы и не получившие первую выплату.

### Проверка данных на аномалии и исправления.

Выведем на экран количество пропусков.

In [3]:
print(data.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Скорее всего столбцы days_employed и total_income взаимосвязаны.

Проверим каждый стобец.

#### Столбец 'children'

In [5]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Некорректные значения в столбце детей. Необходимо убрать отрицательное значение (скорее всего опечатка), а также где написано что в семье 20 детей.

#### Столбец 'days_employed'

In [6]:
print(data['days_employed'].describe())

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64


В данных присутствуют слишком большие значения, также имеется наличие отрицательных.
На дальнейшие исследования не влияет. Поэтому можно его не трогать.

#### Столбец 'dob_years'

In [7]:
print(data['dob_years'].value_counts().sort_index())

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64


'0' - неверное значение. Либо забыли заполнить, либо какая-то ошибка. В целом это малая доля от общего количества заемщиков - на исследования влиять не будет. Лучше их просто убрать.

#### Столбцы 'education' и 'education_id'

In [8]:
print(data['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


In [9]:
print(data['education_id'].value_counts())

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64


В стобцах встречаются дубликаты из-за регистров. Необходимо привести все значения к общему регистру.

#### Столбцы family_status и family_status_id

In [10]:
print(data['family_status'].value_counts())
print(data['family_status_id'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


Здесь все совпадает. Столбцы корректны.

#### Столбец 'gender'

In [11]:
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


Всего одно неопределенное значение. Добавим его к большему количеству полов.

#### Столбец 'income_type'

In [12]:
print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64


Со столбцом все впорядке. Оставим без изменений.

#### Столбец 'debt'

In [13]:
print(data['debt'].value_counts())

0    19784
1     1741
Name: debt, dtype: int64


Все корректно. Кредит либо есть, либо его нет.

#### Столбец 'total_income'

In [14]:
print(data['total_income'].describe())

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: total_income, dtype: float64


In [15]:
display(data[data['total_income'].isnull()].head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Имеются люди без поступления дохода. Данный столбец важен для решения задачи. Необходимо решить проблему.

#### Столбец 'purpose'

In [16]:
print(data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Много совпадений по целям кредита. Необходимо похожие значения привести к одному виду.

### Обработка пропусков

#### Пропуски в столбцах 'days_employed' и 'total_income'.

In [17]:
missed_values = data.loc[(data['days_employed'].isnull()) & (data['total_income'].isnull()),'dob_years'].count()
print(f'Кол-во значений отсутствующих одновременно в обоих столбцах = {missed_values}')
print()
display(data.loc[(data['days_employed'].isnull()) & (data['total_income'].isnull())].head(10))

Кол-во значений отсутствующих одновременно в обоих столбцах = 2174



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Поскольку количество пропусков в них равно, посмотрим верно ли предположение, что это - молодеж, которая еще не работала.

In [18]:
display(data.loc[(data['days_employed'].isnull()) & (data['total_income'].isnull())].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Мы вывели 10 строчек с условием, что столбцы 'total_income' и 'days_employed' одновременно = "NaN".
Видно что возраст варьируется от 21 до 65. Значит предположение ,что это никогда неработавшие - неверно.
Столбец 'total_income' заполним средним значением. А столбец 'days_employed' удалим. Во всех остальных столбцах тоже стоят разные значения. Значит будем заполнять 'total_income' средним значением (median) категории значение в каждой категории. А столбец 'days_employed' удалим.

#### Удаляем столбец 'days_employed'

In [19]:
data.drop('days_employed', axis='columns', inplace=True)
print(data.columns)

Index(['children', 'dob_years', 'education', 'education_id', 'family_status',
       'family_status_id', 'gender', 'income_type', 'debt', 'total_income',
       'purpose'],
      dtype='object')


#### Заполняем столбец 'total_income'

In [20]:
display(data.info())
print()
total_income_median = data['total_income'].median()
display(f'Медиана столбца = {total_income_median}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


None

'Медиана столбца = 145017.93753253992'

In [21]:
income_category_list = data.groupby('income_type')['total_income'].median().sort_values()
print('Таблица медиан по категориям : ')
display(income_category_list)
print()
print('Пропущеных значений всего в total_income - ', data['total_income'].isna().sum())
print("Выведем несколько строк с пропусками:")
display(data.loc[data.loc[:,'total_income'].isna()==True].head(10))

Таблица медиан по категориям : 


income_type
в декрете           53829.130729
студент             98201.625314
пенсионер          118514.486412
безработный        131339.751676
сотрудник          142594.396847
госслужащий        150447.935283
компаньон          172357.950966
предприниматель    499163.144947
Name: total_income, dtype: float64


Пропущеных значений всего в total_income -  2174
Выведем несколько строк с пропусками:


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [22]:
for category in income_category_list.index:
    median = income_category_list[category]
    print(category,': Медиана = ', median)
    print(median)
    print('Пропусков было:', data[data['income_type']==category]['total_income'].isnull().sum())
    
    #data['total_income'] = data[data['income_type']==category]['total_income'].fillna(median)
    #data[data['income_type']==category]['total_income'] = data[data['income_type']==category]['total_income'].fillna(median)
    
    #data['total_income'] = data.loc[data.loc[:, 'income_type']==category, 'total_income'].fillna(median)
    data.loc[data.loc[:, 'income_type']==category, 'total_income'] = data.loc[data.loc[:, 'income_type']==category, 'total_income'].fillna(median)
    
    print('Пропусков стало:', data[data['income_type']==category]['total_income'].isnull().sum())
    print()
    
print('Осталось пропущеных значений - ', data['total_income'].isna().sum())
print()

в декрете : Медиана =  53829.13072905995
53829.13072905995
Пропусков было: 0
Пропусков стало: 0

студент : Медиана =  98201.62531401133
98201.62531401133
Пропусков было: 0
Пропусков стало: 0

пенсионер : Медиана =  118514.48641164352
118514.48641164352
Пропусков было: 413
Пропусков стало: 0

безработный : Медиана =  131339.7516762103
131339.7516762103
Пропусков было: 0
Пропусков стало: 0

сотрудник : Медиана =  142594.39684740017
142594.39684740017
Пропусков было: 1105
Пропусков стало: 0

госслужащий : Медиана =  150447.9352830068
150447.9352830068
Пропусков было: 147
Пропусков стало: 0

компаньон : Медиана =  172357.95096577113
172357.95096577113
Пропусков было: 508
Пропусков стало: 0

предприниматель : Медиана =  499163.1449470857
499163.1449470857
Пропусков было: 1
Пропусков стало: 0

Осталось пропущеных значений -  0



In [23]:
print("Для проверки выведем строки где были заменены пропуски из разных категорий № 12,26,65,82")
print(data.loc[[12,26,65,82], ['total_income', 'income_type']])
print()
print('Таблица медиан по категориям : ')
print(income_category_list)
print()
print("Все медианы по категориям совпадают.")
#print(data.loc[data.loc[:,'total_income'].isna()==True].head(10))

Для проверки выведем строки где были заменены пропуски из разных категорий № 12,26,65,82
     total_income  income_type
12  118514.486412    пенсионер
26  150447.935283  госслужащий
65  172357.950966    компаньон
82  142594.396847    сотрудник

Таблица медиан по категориям : 
income_type
в декрете           53829.130729
студент             98201.625314
пенсионер          118514.486412
безработный        131339.751676
сотрудник          142594.396847
госслужащий        150447.935283
компаньон          172357.950966
предприниматель    499163.144947
Name: total_income, dtype: float64

Все медианы по категориям совпадают.


#### Столбец 'gender'

Исправим единственное значение "XNA" на "F"-женщина

In [24]:
data.loc[data['gender']=='XNA']
data.loc[data['gender']=='XNA','gender'] = 'F'
print(data.loc[10701])

children                               0
dob_years                             24
education            неоконченное высшее
education_id                           2
family_status           гражданский брак
family_status_id                       1
gender                                 F
income_type                    компаньон
debt                                   0
total_income                      203905
purpose             покупка недвижимости
Name: 10701, dtype: object


#### Столбец 'children'

In [25]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Посмотрим как выглядят строки с значениями 20 и -1.

In [26]:
display(data[data['children']==-1].head(5))

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,57,Среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,на покупку своего автомобиля


In [27]:
display(data[data['children']==20].head(5))

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля


Явной закономерности не прослеживается. Скорее всего опечатка, либо ошибка при получении данных. 
Верный вариант будет убрать эти строчки т.к. их мало и они не должны повлиять на конечный результат.

In [28]:
index_1 = data[data['children']==-1].index # индексы 47 строк где значение -1
index_2 = data[data['children']==20].index # индексы 76 строк где значение 20 

str_del_count=0
try:
    for drop_str in index_1:
        data = data.drop(drop_str)
        str_del_count+=1
except:
    print('Неполучилось удалить строку')
print('Кол-во удаленных строк c "-1" =', str_del_count)

str_del_count=0
try:
    for drop_str in index_2:
        data = data.drop(drop_str)
        str_del_count+=1
except:
    print('Неполучилось удалить строку')    
print('Кол-во удаленных строк c "20" =', str_del_count)
print('')
print('')
data = data.reset_index(drop=True)
print(data.count())
print()
print()
print('Должно остаться строк =', 21525-47-76)

Кол-во удаленных строк c "-1" = 47
Кол-во удаленных строк c "20" = 76


children            21402
dob_years           21402
education           21402
education_id        21402
family_status       21402
family_status_id    21402
gender              21402
income_type         21402
debt                21402
total_income        21402
purpose             21402
dtype: int64


Должно остаться строк = 21402


In [29]:
# Код ревьюера
# df = df[(df['children'] != -1)&(df['children'] != 20)]

#### Столбец dob_years

Количество значений, которые надо корректировать не так велико, относительно массива данных. В исследовании стобец не участвует. Можно не вносить коррективы.

In [30]:
print(data['dob_years'].value_counts().sort_index())

0     100
19     14
20     51
21    110
22    183
23    252
24    263
25    356
26    406
27    490
28    501
29    543
30    536
31    556
32    506
33    577
34    597
35    614
36    553
37    531
38    595
39    572
40    603
41    603
42    592
43    510
44    543
45    494
46    469
47    480
48    536
49    505
50    509
51    446
52    483
53    457
54    476
55    441
56    482
57    457
58    461
59    441
60    376
61    353
62    351
63    268
64    263
65    194
66    183
67    167
68     99
69     83
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64


In [31]:
display(data[data['dob_years']==0].head(20))

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
577,0,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1033,0,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1141,0,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1167,0,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1377,0,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1881,0,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,142594.396847,жилье
1889,0,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


In [32]:
#print('Размер датафрейма до :', data.shape)
#index_years = data[data['dob_years']==0].index #индексы строк необходимых для удаления
#for drop_str in index_years:
#    data = data.drop(drop_str)
#print('Размер датафрейма после :', data.shape)

### Изменение типов данных.

Заменим тип в 2 столбцац: 
1) debt на bool
2) total_income на int

In [33]:
data['debt'] = data['debt'].astype(bool)

data['total_income'] = data['total_income'].astype(int)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21402 entries, 0 to 21401
Data columns (total 11 columns):
children            21402 non-null int64
dob_years           21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
income_type         21402 non-null object
debt                21402 non-null bool
total_income        21402 non-null int64
purpose             21402 non-null object
dtypes: bool(1), int64(5), object(5)
memory usage: 1.7+ MB


Тип данных заменили. Теперь можно с ними спокойно работать.

### Удаление дубликатов.

#### Столбец 'education'

Получим информацию по столбцу методом .value_counts()

In [34]:
print(data['education'].value_counts(), data['education_id'].value_counts())

среднее                13667
высшее                  4698
СРЕДНЕЕ                  766
Среднее                  703
неоконченное высшее      665
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64 1    15136
0     5237
2      741
3      282
4        6
Name: education_id, dtype: int64


Как мы видим, проблема в регистре. Необходимо привести все наименования к общему регистру.

In [35]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())

среднее                15136
высшее                  5237
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64


#### Столбец 'purpose'

Неоходимо похожие цели привести к общему названию.
Для начала необходимо создать список с значениями для замены.
Необходимо будет с помощью лемматизации изменить некорректные значения.

In [36]:
print(data['purpose'].value_counts())

свадьба                                   796
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
жилье                                     642
покупка жилья                             641
покупка жилья для семьи                   640
недвижимость                              632
строительство собственной недвижимости    628
операции со своей недвижимостью           626
строительство жилой недвижимости          622
строительство недвижимости                620
покупка недвижимости                      619
покупка своего жилья                      619
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля              504
заняться высшим образованием      

Судя по всему данные вводились в ручную, разными людьми. Сгруппируем схожие цели, создадим цикл лематизации и словарь для этого цикла.

In [37]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

columns = ['original', 'lemms', 'good']
purpose_list = pd.DataFrame(data=[],columns=columns)
purpose_list['original'] = data['purpose'].value_counts().index

for i in purpose_list.index:
    purpose_list.loc[i,'lemms'] = ' '.join(m.lemmatize(purpose_list.loc[i, 'original']))
display(purpose_list)

,original,lemms,good
0,свадьба,свадьба \n,NaN
1,на проведение свадьбы,на проведение свадьба \n,NaN
2,сыграть свадьбу,сыграть свадьба \n,NaN
3,операции с недвижимостью,операция с недвижимость \n,NaN
4,покупка коммерческой недвижимости,покупка коммерческий недвижимость \n,NaN
5,покупка жилья для сдачи,покупка жилье для сдача \n,NaN
6,операции с жильем,операция с жилье \n,NaN
7,операции с коммерческой недвижимостью,операция с коммерческий недвижимость \n,NaN
8,жилье,жилье \n,NaN
9,покупка жилья,покупка жилье \n,NaN


In [38]:
dict_lem_purpose = dict({'свадьба':'проведение свадьбы', 
                    'жилье':'операции с недвижимостью', 
                    'недвижимость':'операции с недвижимостью',
                    'автомобиль':'операции с автомобилем', 
                    'образование':'получение образования'})

Нужна функция, которая будет менять значение столбца, на значение из словаря выше, используя цикл который мы написали выше.

In [39]:
def make_good_category_str(row):
    str_lem_list = m.lemmatize(row)
    for lem in str_lem_list:
        if lem in dict_lem_purpose.keys():
            #print(lem, dict_lem_purpose[lem], str_lem_list)
            return dict_lem_purpose[lem]

Применим функцию к столбцу целей.

In [40]:
data['purpose_category'] = data['purpose'].apply(make_good_category_str)

Теперь проверим уникальные значения в "целях" и выведем на экран парочку значений.

In [41]:
print(data['purpose_category'].value_counts())
print()

операции с недвижимостью    10780
операции с автомобилем       4288
получение образования        3997
проведение свадьбы           2337
Name: purpose_category, dtype: int64



По итогам лемматизации у нас теперь 4 цели кредитования. 

недвижимость - 10733
автомобиль - 4267
образование - 3979
свадьба - 2323

In [42]:
# Код ревьюера
data.duplicated().sum()

71

In [43]:
data = data.drop_duplicates()
data.duplicated().sum()

0

## Категоризация данных

Для категоризации подходят два столбца "dob_years" и "total_income".

In [44]:
print(data['total_income'].describe())

count    2.133100e+04
mean     1.653427e+05
std      9.831399e+04
min      2.066700e+04
25%      1.075070e+05
50%      1.425940e+05
75%      1.958420e+05
max      2.265604e+06
Name: total_income, dtype: float64


Чтобы разбить на категории, нужно разделить доход на категории. Для этого разобьем его по процентам и сделаем несколько категорий.

'E': (0–30000); 
'D'(30001–50000); 
'C'(50001–200000); 
'B'(200001–1000000); 
'A'(>1000001); 

In [45]:
print()
for proc in range(2,100,3):
    quant = int(data['total_income'].quantile(proc/100))
    print(f'Процент дохода {proc}% = {quant}')


Процент дохода 2% = 51396
Процент дохода 5% = 64561
Процент дохода 8% = 73243
Процент дохода 11% = 80722
Процент дохода 14% = 86732
Процент дохода 17% = 92788
Процент дохода 20% = 98514
Процент дохода 23% = 103778
Процент дохода 26% = 109386
Процент дохода 29% = 114249
Процент дохода 32% = 118514
Процент дохода 35% = 122507
Процент дохода 38% = 128479
Процент дохода 41% = 133961
Процент дохода 44% = 139465
Процент дохода 47% = 142594
Процент дохода 50% = 142594
Процент дохода 53% = 147211
Процент дохода 56% = 152752
Процент дохода 59% = 159265
Процент дохода 62% = 165499
Процент дохода 65% = 172357
Процент дохода 68% = 174091
Процент дохода 71% = 182837
Процент дохода 74% = 192490
Процент дохода 77% = 202080
Процент дохода 80% = 214604
Процент дохода 83% = 227554
Процент дохода 86% = 243851
Процент дохода 89% = 262404
Процент дохода 92% = 287071
Процент дохода 95% = 331767
Процент дохода 98% = 421901


In [46]:
data['income_rating'] = 'NaN'
data['income_rating_id'] = 'NaN'
data.loc[data['total_income'] < 30000, ['income_rating', 'income_rating_id']] = ['E', 1]
data.loc[(30001 <= data['total_income']) & ( data['total_income'] < 50000), ['income_rating', 'income_rating_id']] = ['D', 2]
data.loc[(50001 <= data['total_income']) & ( data['total_income'] < 200000), ['income_rating', 'income_rating_id']] = ['C', 2]
data.loc[(200001 <= data['total_income']) & ( data['total_income'] < 1000000), ['income_rating', 'income_rating_id']] = ['B', 2]
data.loc[(1000001 <= data['total_income']), ['income_rating', 'income_rating_id']] = ['A', 2]
display(data.head())
data.info()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_rating,income_rating_id
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья,операции с недвижимостью,B,2
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля,операции с автомобилем,C,2
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья,операции с недвижимостью,C,2
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование,получение образования,B,2
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу,проведение свадьбы,C,2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 21331 entries, 0 to 21401
Data columns (total 14 columns):
children            21331 non-null int64
dob_years           21331 non-null int64
education           21331 non-null object
education_id        21331 non-null int64
family_status       21331 non-null object
family_status_id    21331 non-null int64
gender              21331 non-null object
income_type         21331 non-null object
debt                21331 non-null bool
total_income        21331 non-null int64
purpose             21331 non-null object
purpose_category    21331 non-null object
income_rating       21331 non-null object
income_rating_id    21331 non-null int64
dtypes: bool(1), int64(6), object(7)
memory usage: 2.3+ MB


Разбили доходы на 5 категорий.
Добавили рейтинг с обозначением от "А" до "С", в отдельно созданном столбце.

### Ответы на вопросы.

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?:

Составим сводную таблицу методом .pivot_table. Необходимые столбцы - 'children' и 'debt'.

In [47]:
data_children = data.pivot_table( columns='children', values='debt', aggfunc='sum')
display(data_children)

children,0,1,2,3,4,5
debt,1063.0,444.0,194.0,27.0,4.0,0.0


Собственно наглядно видно, что чем больше детей, тем меньше просрочек по платежам. Вероятнее всего наличие детей дисциплинирует и самих родителей.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [48]:
data_family_status = data.groupby(['family_status_id', 'family_status'])['debt'].sum()
print(data_family_status)

family_status_id  family_status        
0                 женат / замужем          927.0
1                 гражданский брак         385.0
2                 вдовец / вдова            63.0
3                 в разводе                 84.0
4                 Не женат / не замужем    273.0
Name: debt, dtype: float64


Выше всего просрочки у того кто женат или замужем. Наименьшие просрочки у людей в статусе вдовы/вдовца.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [49]:
data_income_rating = data.groupby(['income_rating_id', 'income_rating'])['debt'].sum()
print(data_income_rating)

income_rating_id  income_rating
1                 E                   2.0
2                 A                   2.0
                  B                 354.0
                  C                1353.0
                  D                  21.0
Name: debt, dtype: float64


Просрочки чаще всего встречаются у населения со средним кредитным рейтингом. У малого и высокого рейтинга в целом картина более приемлемая.

#### Как цели кредита влияют на возврат в срок?

In [50]:
data_purpose = data.groupby(['purpose_category']).agg({'debt':['sum', 'count']})
data_purpose[('debt', 'part,%')] = data_purpose[('debt', 'sum')] / data_purpose[('debt', 'count')]*100
print(data_purpose)

                           debt                 
                            sum  count    part,%
purpose_category                                
операции с автомобилем    400.0   4279  9.347978
операции с недвижимостью  780.0  10751  7.255139
получение образования     369.0   3988  9.252758
проведение свадьбы        183.0   2313  7.911803


Больше всего просрочек по кредитам в категории "недвижимость". Но в процентном соотношении "автомобили" и "образование" лидируют.
Недвижимость скорее всего лидирует из-за ипотек. Большие платежи на очень большой срок имеют большую вероятность быть не выплаченными в срок.

## Общий вывод:

Поставленные задачи выполнены. В ходе исследования мы проанализировали исходные данные. Нашли все пропуски, дубликаты, ошибки. Лемматизировали данные. Привели все к виду, необходимому для исследовательской работы. 
На основе полученных данных ответили на все поставленные вопросы вопросы и нашли закономерности, там где они имеются.